# Задание 2.1
Напишите запрос для расчета общего объема продаж по каждому продукту (с учетом количества продукта, его цены по прейскуранту и скидки).

(Представлены два способа решения)

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").getOrCreate()


API

In [ ]:
from pyspark.sql import functions as F

df_products = spark.read.csv("CVS\\products.csv", inferSchema=True, header=True, sep=",")
df_order_items = spark.read.csv("CVS\\order_items.csv", inferSchema=True, header=True, sep=",")

df_res = df_products.join(df_order_items, 
                           df_order_items["product_id"] == df_products["product_id"], "inner")


df_res = df_res.groupBy("product_name"
                        ).agg(F.sum
                        (df_order_items["quantity"] * 
                        (df_order_items["list_price"] * 
                         (1-df_order_items["discount"]))
                         ).alias("Total_sales"))

df_res.show()

+--------------------+------------------+
|        product_name|       Total_sales|
+--------------------+------------------+
|Electra Straight ...|          362.6907|
|Electra Heartchya...|         1929.5397|
|Trek Precaliber 2...|255.99200000000002|
|Electra Townie Co...|1646.9816999999998|
|Electra Townie Ba...|6497.9277999999995|
|Electra Tiger Sha...|1804.7435999999998|
|Trek Stache Carbo...|4149.1548999999995|
|Heller Bloodhound...|           8992.54|
|Trek Precaliber 2...| 640.0826999999999|
|Electra Townie Or...|1574.9650000000001|
|Electra Townie Or...|          4963.927|
|Electra Amsterdam...|3035.9746999999998|
|Electra Townie Or...|29164.204799999996|
|Trek Slash 8 27.5...|       555558.6111|
|Trek Boy's Kickst...|         3236.7842|
|Electra Townie Or...|2069.9539999999997|
|Sun Bicycles Spid...|19491.966000000004|
|Trek Remedy 7 27....|14069.953099999999|
|Trek Domane ALR 4...| 9749.437100000001|
|Trek Farley Carbo...|         7239.9276|
+--------------------+------------

spark.sql

In [5]:
df_products.createOrReplaceTempView("products")
df_order_items.createOrReplaceTempView("order_items")

spark.sql("""
            Select  products.product_name , (SUM(order_items.quantity * (order_items.list_price * (1-order_items.discount)))) as total_sales
            FROM products
            Inner Join order_items on order_items.product_id = products.product_id
            Group by products.product_name
          """).show()

+--------------------+------------------+
|        product_name|       total_sales|
+--------------------+------------------+
|Electra Straight ...|          362.6907|
|Electra Heartchya...|         1929.5397|
|Trek Precaliber 2...|255.99200000000002|
|Electra Townie Co...|1646.9816999999998|
|Electra Townie Ba...|6497.9277999999995|
|Electra Tiger Sha...|1804.7435999999998|
|Trek Stache Carbo...|4149.1548999999995|
|Heller Bloodhound...|           8992.54|
|Trek Precaliber 2...| 640.0826999999999|
|Electra Townie Or...|1574.9650000000001|
|Electra Townie Or...|          4963.927|
|Electra Amsterdam...|3035.9746999999998|
|Electra Townie Or...|29164.204799999996|
|Trek Slash 8 27.5...|       555558.6111|
|Trek Boy's Kickst...|         3236.7842|
|Electra Townie Or...|2069.9539999999997|
|Sun Bicycles Spid...|19491.966000000004|
|Trek Remedy 7 27....|14069.953099999999|
|Trek Domane ALR 4...| 9749.437100000001|
|Trek Farley Carbo...|         7239.9276|
+--------------------+------------